# Check your installation

This notebook is meant to check if your infrastructure is supporting the requirements of this course. Since you are currently running this notebook, we guess you got the file with `git` !

In [20]:
!git --version

git version 2.39.2


## Xeus-cling C++17 kernel

If you have the relevant kernel installed, you should be able to execute the following code cells.

In [21]:
#include <iostream>

In [22]:
{
  int i = -1 ;
  unsigned int j = 1 ;
  if ( i < j ) std::cout << "i is less than j" ;
  else std::cout << "i is greater than j" ;
}

i is greater than j

## G++ 11.2

The more we progress in the lessons, the more advanced C++ features we will use: mostly C++17, and C++20 for few demonstrations. Even in the notebooks, when we want to practice the recent C++20 features, or the few C++17 features that are not supported by the xeus-cling notebook kernel, we will generate a file on-the-fly and compile it directly with g++. We currently recommend to use at least g++ 11.2.

In [23]:
!g++ --version

g++ (GCC) 13.2.0
Copyright (C) 2023 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



Note: you will find [here](https://en.cppreference.com/w/cpp/compiler_support) the table of which C++ feature is supported by which compiler version. You should be aware that there is often a distinction between language syntax and standard library : a compiler version which supports C++20 language features does not necessarily supports the complete C++20 standard library. Generally, the support for syntax is implemented first, and the library novelties comes progressively with later versions.

## Gnu time

For some of the exercices, you will need to have **GNU time** installed where you want to execute the scripts. Try the command `\time -f "%U" echo toto` to check if it works fine. The `\` is there to avoid that bash use its own internal time command. If it does not work properly, try `conda install -c conda-forge time` (for example). 

In [24]:
!\time -f "%U" echo 42

42
0.00
